In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as ipw
#from ipywidgets import AppLayout, Button, Layout
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

import numpy as np
import time


Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Langmuir isotherm virtual lab

The Langmuir isotherm is one of the simplest models that can be used to describe the adsorption of molecules on surfaces, either in the gas phase or in solutions. Refer to the lectures or to online resources for a discussion of the assumptions and limitation of this model.
The fundamental equation of the Langmuir adsorption isotherm is

\begin{equation}
\theta = \frac{Kc}{1+Kc}
\end{equation}

where $\theta$ is the fraction of adsorption sites that are occupied, $K$ is the Langmuir equilibrium constant, and $c$ is the equilibrium concentration of the adsorbate in solution.
Because it is not possible to directly measure the fraction of occupied surface sites, a more practical version of that equation is

\begin{equation}
c_s = \frac{QKc}{1+Kc}
\end{equation}

where $c_s$ is the concentration of the adsorbate that is on the surface, _i.e._ out of the solution, and the new parameter $Q$ corresponds to the _monolayer_ coverage, _i.e._ the maximum concentration of molecules that can adsorb on the substrate. The linear form of the above equation, which uses the inverse of the concentrtions, is more convenient for the fitting; deriving it is left to you as part of this assignment.

The name _isotherm_ stems from the fact that the experiments are performed at constant temperature and in principle both the Langmuir equilibrium constant, $K$, and the _monolayer_ coverage, $Q$, can have a temperature dependence.
Similarly to normal chemical reactions, by performing a series of experiments at different conditions it is possible to determine the enthalpy and entropy of the adsorption process using the van't Hoff equation.

In the virtual laboratory below, you will be looking at the adsorption of the dye Acid Blue 158 on chitin in water. Perform a series of experiments at different conditions to determine the enthalpy of adsorption of the dye on the substrate. The molar mass of Acid Blue 158 is 584.91 g/mol.

### Instructions
* Use your student ID to initialise the lab and generate your unique data set
* Select the temperature and amount on water for your experiments
* Select an appropriate minimum and maximum amount of acid Blue 158 to use in the experiments; this has to cover a large enough range to allow for a proper fit of the curve.
* Select how many experiments you want to perform ($N$)

The code will then generate a $N$ observations at the selected temperature where the amount of dye that is added to  the chosen amount of water, which contains the chitin powder, is varied between the minimum and maximum values that you have chosen, in equally spaced intervals.

Every time you click "Perform experiment" $N$ new observations will be generated and appended to the output file.

If you click "Reset experiments" all observations will be deleted.
This will be useful to generate a clean set of data after you have done a few tests to find what is an appropriate range for the amount of dye to use in the virtual experiment.

You can download all the observations in CSV format and import them directly into excel.


In [3]:
# define path to results.csv file
respath = os.path.join(os.getcwd(), "..", "results.csv")

# delete existing result file and setup rng
if os.path.exists(respath):
    os.remove(respath)

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params = {}

delta = 0.2


In [4]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

def measure(K,Q,T,V,mass):
    # concentration in mol/L
    c0 = mass / params["molarMass"] / V

    # compute the concentration of dye in solution
    res = ((c0*K - K*Q - 1) + math.sqrt((c0*K - K*Q - 1)**2 + 4*c0*K) ) / (2*K)
    
    return res
    
def calc(btn):
#     if os.path.exists(respath):
#         os.remove(respath)
#     res = pd.DataFrame(columns=["Temperature [C]", "Dye added [mg]", "Dye in solution [M]"])
#     res.to_csv(respath, index=False)
#     with out_P:
#         out_P.clear_output()
#         display(res.tail(10))
    out_P.clear_output()


    temp = float(key1.value) + 273.15 # from C to K

    random.seed( int(key0.value) )
    rnd1 = (2*random.random() - 1) * 0.2
    rnd2 = (2*random.random() - 1) * 0.2
    rnd3 = (2*random.random() - 1) * 0.2
    dH = params["dH"] * (1.+rnd1)
    dS = params["dS"] * (1.+rnd2)
    Q = params["Q"] * (1.+rnd3) # in mol/L    
    
    lnK = (-dH / temp + dS) / 0.0083145

    K = math.exp(lnK) # in L/mol
#     print(dH,dS,K,Q)

    V = float(keyV.value)
    minDye = float(key2.value) # input in mg
    maxDye = float(key3.value) # input in mg
    nsteps = int(key4.value)
    if nsteps == 1:
        deltaMass = 0
    else:
        deltaMass = (maxDye-minDye) / (nsteps-1)

    for istep in range(0,nsteps):
        massDye = minDye + deltaMass * float(istep)

    # Measurement result
        exact = measure(K,Q,temp,V,massDye/1000)
        result = random.gauss(exact, exact*0.0001)

    # Read previous lines
        res = pd.read_csv(respath) 

        var_list = []
        var_list.append(temp-273.15)            
        var_list.append(V)    
        var_list.append(massDye)            
        var_list.append(result)      
        res.loc[len(res)] = var_list
    
        res.to_csv(respath, index=False)
        
    with out_P:
        display(res.tail(10))

def reset(btn):
    if os.path.exists(respath):
        os.remove(respath)

    # doi.org/10.1002/app.1982.070270827
    params["dH"] = -10 # kJ/mol
    params["dS"] = 0.001 # kJ/mol/K
    params["Q"] = 0.05 # [mol/L]
#     params["Volume"] = 0.1 # L
    params["molarMass"] = 584.910641 # g/mol

    res = pd.DataFrame(columns=["Temperature [C]", "Volume [L]", "Dye added [mg]", "Dye in solution [mol/L]"])
    res.to_csv(respath, index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(10))

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# ---
reset(btn_reset)

# --- create the boxes and sliders
rows = []

label_layout = ipw.Layout(width='300px')

key0 = ipw.Text("123456789")
box00 = ipw.Box([ipw.Label('Student ID  :  ',layout=label_layout),key0])
rows.append(ipw.HBox([box00]))

key1 = ipw.Text("25")
box01 = ipw.Box([ipw.Label('Temperature [C]  :  ',layout=label_layout),key1])
rows.append(ipw.HBox([box01]))

keyV = ipw.Text("0.1")
boxV = ipw.Box([ipw.Label('Volume [L]  :  ',layout=label_layout),keyV])
rows.append(ipw.HBox([boxV]))

key2 = ipw.Text("50")
box02 = ipw.Box([ipw.Label('Minimum dye amount [mg]  :  ',layout=label_layout),key2])
rows.append(ipw.HBox([box02]))

key3 = ipw.Text("100")
box03 = ipw.Box([ipw.Label('Maximum dye amount [mg]  :  ',layout=label_layout),key3])
rows.append(ipw.HBox([box03]))

key4 = ipw.Text("2")
box04 = ipw.Box([ipw.Label('Number of experiments  :  ',layout=label_layout),key4])
rows.append(ipw.HBox([box04]))


rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)